# Algorytmy tekstowe 2019/2020

# Laboratorium 4

# Autor - Łukasz Jezapkowicz

# Laboratorium dotyczy wykorzystania odległości edycyjnej.

# 1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

Na potrzeby laboratorium zaimplementuję znany algorytm wykorzystujący strategię programowania dynamicznego (DP). Algorytm ten oblicza odległość edycyjną (inaczej zwaną odległością Levenshteina) oraz po odpowiednim przygotowaniu potrafi tą sekwencję zwrócić użytkownikowi. Nasz algorytm posiada złożoność obliczeniową czasową O(a*b), gdzie a i b to długości porównywanych słów zaś złożoność pamięciową O(a) (lub O(b), zależy jak się zakoduje). Jest to możliwe do osiągnięcia poprzez "przesuwanie" wierszami w dół.

In [31]:
# metoda do obliczania odległości edycyjnej używająca O(a) pamięci
def edit_distance(s,t):
    a = len(s)
    b = len(t)
    
    dist = [[0] * (a+1) for i in range(2)]
    for i in range(a+1):
        dist[0][i] = i
    
    for i in range(1,b+1):
        for j in range(a+1):
            if j == 0:
                dist[i%2][j] = i
            elif s[j-1] == t[i-1]:
                dist[i%2][j] = dist[(i-1)%2][j-1]
            else:
                dist[i%2][j] = 1 + min(min(dist[i%2][j-1],dist[(i-1)%2][j-1]),dist[(i-1)%2][j])
    
    return dist[b%2][a]

Niestety taka implementacja mimo zaoszczędzonej pamięci utrudnia nam znalezienie sekwencji zmian potrzebnych do przekształcenia słowa s w słowo t. Potrzebna jest nam więc implementacja zajmująca więcej pamięci (to jest O(a*b))

In [32]:
# metoda do obliczania odległości edycyjnej używająca O(a*b) pamięci
def edit_distance_better(s,t):
    a = len(s)
    b = len(t)
    
    dist = [[0] * (b+1) for i in range(a+1)]
    
    for i in range(a+1):
        for j in range(b+1):
            if i == 0:
                dist[i][j] = j
            elif j == 0:
                dist[i][j] = i
            elif s[i-1] == t[j-1]:
                dist[i][j] = dist[i-1][j-1]
            else:
                dist[i][j] = 1 + min(min(dist[i][j-1],dist[i-1][j-1]),dist[i-1][j])
    
    return dist[a][b], dist

Możemy zobaczyć równowartość wyników dla kilku par słów:

In [33]:
print(edit_distance("Algorytmy","tekstowe"),edit_distance_better("Algorytmy","tekstowe")[0])
print(edit_distance("Avengers","Marvel"),edit_distance_better("Avengers","Marvel")[0])
print(edit_distance("Koronawirus","Korona piwo"),edit_distance_better("Koronawirus","Korona piwo")[0])
print(edit_distance("Nienawidzę","poniedziałków"),edit_distance_better("Nienawidzę","poniedziałków")[0])
print(edit_distance("Makumba","Makumba"),edit_distance_better("Makumba","Makumba")[0])

9 9
7 7
5 5
11 11
0 0


# 2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym.

Teraz zaimplementuję funkcję pokazującą jak ze słowa s zrobić słowo t krok po kroku:

In [34]:
# metoda wypisująca sekwencję zmian prowadzących do zamiany słowa s w słowo t
def get_editorial_sequence(s,t):
    sequence = []
    m,n = len(s),len(t)
    
    dist = edit_distance_better(s,t)[1]
    print("Ilość kroków: " + str(dist[m][n]))
    
    # 1 - zamiana litery
    # 2 - usunięcie litery
    # 3 - dodanie litery
    while m != 0 or n != 0:
        min_i,min_j = m-1,n-1
        if dist[min_i][min_j] > dist[m-1][n]:
            min_i,min_j = m-1,n
        if dist[min_i][min_j] > dist[m][n-1]:
            min_i,min_j = m,n-1
        
        if min_i == m-1 and min_j == n-1:
            if dist[min_i][min_j] != dist[m][n]:
                sequence.append((1,m,n))
            m,n = m-1,n-1
        elif min_i == m-1 and min_j == n:
            sequence.append((2,m,n))
            m -= 1
        else:
            sequence.append((3,m,n))
            n -= 1
        
    return print_sequence(s,t,sequence)

# metoda wypisującą kolejne kroki zamiany słowa s w słowa t w przyjemny dla oka sposób
def print_sequence(s,t,sequence):
    print("Zamiana słowa \"" + s + "\" w słowo \"" + t + "\" odbywa się w następujący sposób:")
    new_s = list(s)
    for (i,j,k) in sequence:
        if i == 1:
            print("".join(new_s),end='')
            new_s[j-1] = t[k-1]
            print(" -> Zamiana litery " + s[j-1] + " w literę " + t[k-1] + " -> " + "".join(new_s))
        elif i == 2:
            print("".join(new_s),end='')
            del new_s[j-1]
            print(" -> Usunięcie litery " + s[j-1] + " -> " + "".join(new_s))
        else:
            print("".join(new_s),end='')
            new_s.insert(j,t[k-1])
            print(" -> Dodanie litery " + t[k-1] + " -> " + "".join(new_s))
    
    print(t + " -> oczekiwane")
    print("".join(new_s) + " -> uzyskane")

# 3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
# los - kloc
# Łódź - Lodz
# kwintesencja - quintessence
# ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [35]:
get_editorial_sequence("los","kloc")

Ilość kroków: 2
Zamiana słowa "los" w słowo "kloc" odbywa się w następujący sposób:
los -> Zamiana litery s w literę c -> loc
loc -> Dodanie litery k -> kloc
kloc -> oczekiwane
kloc -> uzyskane


In [36]:
get_editorial_sequence("Łódź","Lodz")

Ilość kroków: 3
Zamiana słowa "Łódź" w słowo "Lodz" odbywa się w następujący sposób:
Łódź -> Zamiana litery ź w literę z -> Łódz
Łódz -> Zamiana litery ó w literę o -> Łodz
Łodz -> Zamiana litery Ł w literę L -> Lodz
Lodz -> oczekiwane
Lodz -> uzyskane


In [37]:
get_editorial_sequence("kwintesencja","quintessence")

Ilość kroków: 5
Zamiana słowa "kwintesencja" w słowo "quintessence" odbywa się w następujący sposób:
kwintesencja -> Zamiana litery a w literę e -> kwintesencje
kwintesencje -> Usunięcie litery j -> kwintesence
kwintesence -> Dodanie litery s -> kwintessence
kwintessence -> Zamiana litery w w literę u -> kuintessence
kuintessence -> Zamiana litery k w literę q -> quintessence
quintessence -> oczekiwane
quintessence -> uzyskane


In [38]:
get_editorial_sequence("ATGAATCTTACCGCCTCG","ATGAGGCTCTGGCCCCTG")

Ilość kroków: 7
Zamiana słowa "ATGAATCTTACCGCCTCG" w słowo "ATGAGGCTCTGGCCCCTG" odbywa się w następujący sposób:
ATGAATCTTACCGCCTCG -> Usunięcie litery C -> ATGAATCTTACCGCCTG
ATGAATCTTACCGCCTG -> Zamiana litery G w literę C -> ATGAATCTTACCCCCTG
ATGAATCTTACCCCCTG -> Zamiana litery C w literę G -> ATGAATCTTAGCCCCTG
ATGAATCTTAGCCCCTG -> Zamiana litery A w literę G -> ATGAATCTTGGCCCCTG
ATGAATCTTGGCCCCTG -> Dodanie litery C -> ATGAATCTCTGGCCCCTG
ATGAATCTCTGGCCCCTG -> Zamiana litery T w literę G -> ATGAAGCTCTGGCCCCTG
ATGAAGCTCTGGCCCCTG -> Zamiana litery A w literę G -> ATGAGGCTCTGGCCCCTG
ATGAGGCTCTGGCCCCTG -> oczekiwane
ATGAGGCTCTGGCCCCTG -> uzyskane


BONUSOWO:

In [39]:
get_editorial_sequence("Algorytmy","tekstowe")

Ilość kroków: 9
Zamiana słowa "Algorytmy" w słowo "tekstowe" odbywa się w następujący sposób:
Algorytmy -> Zamiana litery y w literę e -> Algorytme
Algorytme -> Zamiana litery m w literę w -> Algorytwe
Algorytwe -> Zamiana litery t w literę o -> Algoryowe
Algoryowe -> Zamiana litery y w literę t -> Algortowe
Algortowe -> Zamiana litery r w literę s -> Algostowe
Algostowe -> Zamiana litery o w literę k -> Algkstowe
Algkstowe -> Zamiana litery g w literę e -> Alekstowe
Alekstowe -> Zamiana litery l w literę t -> Atekstowe
Atekstowe -> Usunięcie litery A -> tekstowe
tekstowe -> oczekiwane
tekstowe -> uzyskane


In [40]:
get_editorial_sequence("Avengers","Marvel")

Ilość kroków: 7
Zamiana słowa "Avengers" w słowo "Marvel" odbywa się w następujący sposób:
Avengers -> Zamiana litery s w literę l -> Avengerl
Avengerl -> Usunięcie litery r -> Avengel
Avengel -> Zamiana litery g w literę v -> Avenvel
Avenvel -> Zamiana litery n w literę r -> Avervel
Avervel -> Zamiana litery e w literę a -> Avarvel
Avarvel -> Zamiana litery v w literę M -> AMarvel
AMarvel -> Usunięcie litery A -> Marvel
Marvel -> oczekiwane
Marvel -> uzyskane


In [41]:
get_editorial_sequence("Koronawirus","Korona piwo")

Ilość kroków: 5
Zamiana słowa "Koronawirus" w słowo "Korona piwo" odbywa się w następujący sposób:
Koronawirus -> Zamiana litery s w literę o -> Koronawiruo
Koronawiruo -> Zamiana litery u w literę w -> Koronawirwo
Koronawirwo -> Zamiana litery r w literę i -> Koronawiiwo
Koronawiiwo -> Zamiana litery i w literę p -> Koronawpiwo
Koronawpiwo -> Zamiana litery w w literę   -> Korona piwo
Korona piwo -> oczekiwane
Korona piwo -> uzyskane


In [42]:
get_editorial_sequence("Nienawidzę","poniedziałków")

Ilość kroków: 11
Zamiana słowa "Nienawidzę" w słowo "poniedziałków" odbywa się w następujący sposób:
Nienawidzę -> Zamiana litery ę w literę w -> Nienawidzw
Nienawidzw -> Zamiana litery z w literę ó -> Nienawidów
Nienawidów -> Zamiana litery d w literę k -> Nienawików
Nienawików -> Zamiana litery i w literę ł -> Nienawłków
Nienawłków -> Zamiana litery w w literę a -> Nienaałków
Nienaałków -> Zamiana litery a w literę i -> Nieniałków
Nieniałków -> Zamiana litery n w literę z -> Nieziałków
Nieziałków -> Dodanie litery d -> Niedziałków
Niedziałków -> Zamiana litery N w literę n -> niedziałków
niedziałków -> Dodanie litery o -> oniedziałków
oniedziałków -> Dodanie litery p -> poniedziałków
poniedziałków -> oczekiwane
poniedziałków -> uzyskane


In [43]:
get_editorial_sequence("Makumba","Makumba")

Ilość kroków: 0
Zamiana słowa "Makumba" w słowo "Makumba" odbywa się w następujący sposób:
Makumba -> oczekiwane
Makumba -> uzyskane


# 4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

Algorytm obliczania najdłuższego wspólnego podciągu (LSC) jest również algorytmem dynamicznym, w swej budowie bardzo podobnym do algorytmu szukania odległości edycyjnej.

In [44]:
# metoda do obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów
def lcs(s,t):
    a = len(s)
    b = len(t)
    
    lcs_table = [[0] * (b+1) for i in range(a+1)]
    
    for i in range(a+1):
        for j in range(b+1):
            if i == 0 or j == 0:
                lcs_table[i][j] = 0
            elif s[i-1] == t[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i-1][j],lcs_table[i][j-1])
    
    return lcs_table[a][b]

### Przykłady

In [45]:
print("Najdłuższy wspólny podciąg (LCS) słów los oraz kloc wynosi " + str(lcs("los","kloc")))

Najdłuższy wspólny podciąg (LCS) słów los oraz kloc wynosi 2


In [46]:
print("Najdłuższy wspólny podciąg (LCS) słów Łódź oraz Lodz wynosi " + str(lcs("Łódź","Lodz")))

Najdłuższy wspólny podciąg (LCS) słów Łódź oraz Lodz wynosi 1


In [47]:
print("Najdłuższy wspólny podciąg (LCS) słów kwintesencja oraz quintessence wynosi " + str(lcs("kwintesencja","quintessence")))

Najdłuższy wspólny podciąg (LCS) słów kwintesencja oraz quintessence wynosi 8


In [48]:
print("Najdłuższy wspólny podciąg (LCS) słów ATGAATCTTACCGCCTCG oraz ATGAGGCTCTGGCCCCTG wynosi " + 
      str(lcs("ATGAATCTTACCGCCTCG","ATGAGGCTCTGGCCCCTG")))

Najdłuższy wspólny podciąg (LCS) słów ATGAATCTTACCGCCTCG oraz ATGAGGCTCTGGCCCCTG wynosi 13


### Ciekawostka

Przy pomocy algorytmu LCS można obliczyć odległość edycyjną dla przypadku nieuwzględniającego zamianę liter.

In [49]:
# metoda do obliczania odległości edycyjnej dla usuwania oraz dodawania liter
def edit_distance_no_substitution(s,t):
    lcs_value = lcs(s,t)
    return len(s)+len(t)-2*lcs_value

In [50]:
print("Odległość edycyjna bez zamiany liter dla słów los oraz kloc wynosi " 
      + str(edit_distance_no_substitution("los","kloc")))

Odległość edycyjna bez zamiany liter dla słów los oraz kloc wynosi 3


In [51]:
print("Odległość edycyjna bez zamiany liter dla słów Łódź oraz Lodz wynosi " 
      + str(edit_distance_no_substitution("Łódź","Lodz")))

Odległość edycyjna bez zamiany liter dla słów Łódź oraz Lodz wynosi 6


In [52]:
print("Odległość edycyjna bez zamiany liter dla słów kwintesencja oraz quintessence wynosi " 
      + str(edit_distance_no_substitution("kwintesencja","quintessence")))

Odległość edycyjna bez zamiany liter dla słów kwintesencja oraz quintessence wynosi 8


In [53]:
print("Odległość edycyjna bez zamiany liter dla słów ATGAATCTTACCGCCTCG oraz ATGAGGCTCTGGCCCCTG wynosi " 
      + str(edit_distance_no_substitution("ATGAATCTTACCGCCTCG","ATGAGGCTCTGGCCCCTG")))

Odległość edycyjna bez zamiany liter dla słów ATGAATCTTACCGCCTCG oraz ATGAGGCTCTGGCCCCTG wynosi 10


# 5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.

In [54]:
with open("romeo-i-julia.txt",mode='r', encoding="utf-8") as f:
    data = f.read()
f.close()
print(data)

William Shakespeare

Romeo i Julia
tłum. Józef Paszkowski

ISBN 978-83-288-2903-9



OSOBY:
 * ESKALUS — książę panujący w Weronie
 * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
 * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
 * STARZEC — stryjeczny brat Kapuleta
 * ROMEO — syn Montekiego
 * MERKUCJO — krewny księcia
 * BENWOLIO — synowiec Montekiego
 * TYBALT — krewny Pani Kapulet
 * LAURENTY — ojciec franciszkanin
 * JAN — brat z tegoż zgromadzenia
 * BALTAZAR — służący Romea
 * SAMSON, GRZEGORZ — słudzy Kapuleta
 * ABRAHAM — służący Montekiego
 * APTEKARZ
 * TRZECH MUZYKANTÓW
 * PAŹ PARYSA
 * PIOTR
 * DOWÓDCA WARTY
 * PANI MONTEKI — małżonka Montekiego
 * PANI KAPULET — małżonka Kapuleta
 * JULIA — córka Kapuletów
 * MARTA — mamka Julii
 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.




Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.

In [55]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

tokenized_data = [token for token in tokenizer(data) if not token.is_space] # pozbywam się pustych słów

In [56]:
print("Ilość tokenów = " + str(len(tokenized_data)))
for token in tokenized_data:
    print(token)

Ilość tokenów = 27894
William
Shakespeare
Romeo
i
Julia
tłum
.
Józef
Paszkowski
ISBN
978
-
83
-
288
-
2903
-
9
OSOBY
:
*
ESKALUS
—
książę
panujący
w
Weronie
*
PARYS
—
młody
Weroneńczyk
szlachetnego
rodu
,
krewny
księcia
*
MONTEKI
,
KAPULET
—
naczelnicy
dwóch
domów
nieprzyjaznych
sobie
*
STARZEC
—
stryjeczny
brat
Kapuleta
*
ROMEO
—
syn
Montekiego
*
MERKUCJO
—
krewny
księcia
*
BENWOLIO
—
synowiec
Montekiego
*
TYBALT
—
krewny
Pani
Kapulet
*
LAURENTY
—
ojciec
franciszkanin
*
JAN
—
brat
z
tegoż
zgromadzenia
*
BALTAZAR
—
służący
Romea
*
SAMSON
,
GRZEGORZ
—
słudzy
Kapuleta
*
ABRAHAM
—
służący
Montekiego
*
APTEKARZ
*
TRZECH
MUZYKANTÓW
*
PAŹ
PARYSA
*
PIOTR
*
DOWÓDCA
WARTY
*
PANI
MONTEKI
—
małżonka
Montekiego
*
PANI
KAPULET
—
małżonka
Kapuleta
*
JULIA
—
córka
Kapuletów
*
MARTA
—
mamka
Julii
*
Obywatele
weroneńscy
,
różne
osoby
płci
obojej
,
liczący
się
do
przyjaciół
obu
domów
,
maski
,
straż
wojskowa
i
inne
osoby
.
Rzecz
odbywa
się
przez
większą
część
sztuki
w
Weronie
,
przez
część
piątego
aktu


hożych
dziewic
staną
się
tej
nocy
Udziałem
twoim
w
domu
Kapuletów
.
Przyjdź
,
przejrz
i
wybierz
sobie
z
tych
bukietów
Kwiat
najpiękniejszy
.
I
mój
kwiat
tam
luby
Wejdzie
do
liczby
,
choć
nie
do
rachuby
.
Idźmy
.
/
do
Sługi
/
A
wasze
obejdź
w
krąg
Weronę
,
Wynajdź
osoby
tu
wyszczególnione
/
oddaje
mu
papier
/
I
powiedz
każdej
:
że
mój
dom
otworem
Na
ich
usługi
stanie
dziś
wieczorem
.
/
Wychodzą
Kapulet
i
Parys
.
/
SŁUŻĄCY
Mam
wynaleźć
osoby
tu
wyszczególnione
:
to
się
znaczy
według
tego
,
co
tu
napisano
…
A
cóż
tu
napisano
?
Oto
:
że
szewc
ma
pilnować
łokcia
,
a
krawiec
kopyta
;
rybak
pędzla
,
a
malarz
więcierza
.
Jakże
wynajdę
osoby
tu
wyszczególnione
,
kiedy
nie
mogę
wynaleźć
środka
na
wyczytanie
tego
,
co
osoba
pisząca
tu
wyszczególniła
?
Kazano
mi
jednak
;
muszę
się
udać
do
uczonych
.
Oto
jacyś
ichmoście
.
W
samą
porę
nadchodzą
.
/
Wchodzi
Romeo
i
Benwolio
.
/
BENWOLIO
Tak
,
bracie
,
płomień
spędza
się
płomieniem
,
Ból
dawny
nowym
leczy
się
cierpieniem
;
Kręć
się
na
odwrót
,
gdy
mas

z
dawien
dawna
akredytowanych
Stelmachów
wieszczek
.
W
takich
to
przyborach
Co
noc
harcują
po
głowach
kochanków
,
Którzy
natenczas
marzą
o
miłości
;
Albo
po
giętkich
kolanach
dworaków
,
Którzy
natenczas
o
ukłonach
marzą
;
Albo
po
chudych
palcach
adwokatów
,
Którym
się
wtedy
roją
honoraria
;
Albo
po
ustach
romansowych
damul
,
Którym
się
wtedy
marzą
pocałunki
;
Często
atoli
Mab
na
te
ostatnie
Zsyła
przedwczesne
zmarszczki
,
gdy
ich
oddech
Za
bardzo
znajdzie
cukrem
przesycony
.
Czasem
też
wjeżdża
na
nos
dworakowi
:
Wtedy
śnią
mu
się
nowe
łaski
pańskie
;
Czasem
i
księdza
plebana
odwiedzi
,
Gdy
ten
spokojnie
drzemie
,
i
ogonem
Dziesięcinnego
wieprza
w
nos
go
łechce
:
Wtedy
mu
nowe
śnią
się
beneficja
Czasem
wkłusuje
na
kark
żołnierzowi
:
Ten
wtedy
marzy
o
cięciach
i
pchnięciach
,
O
szturmach
,
breszach
,
o
hiszpańskich
klingach
Czy
o
pucharach
,
co
mają
pięć
sążni
;
Wtem
mu
zatrąbi
w
ucho
:
nasz
bohater
Truchleje
,
zrywa
się
,
klnąc
zmawia
pacierz
I
znów
zasypia
.
Taka
jest
Mab
:
ona
,
Ona
t

Widziałbyś
na
niej
rozlany
rumieniec
Po
tym
,
co
z
ust
mych
słyszałeś
tej
nocy
.
Rada
bym
form
się
trzymać
,
rada
cofnąć
To
,
co
wyrzekłam
;
ale
precz
,
udanie
!
Czy
ty
mię
kochasz
?
Wiem
,
że
powiesz
:
tak
jest
;
I
ja
–
ć
uwierzę
;
mimo
przysiąg
jednak
Możesz
mię
zawieść
.
Z
wiarołomstwa
mężczyzn
Śmieje
się
,
mówią
,
Jowisz
.
O
!
Romeo
!
Jeśli
mię
kochasz
,
wyrzecz
to
rzetelnie
;
Lecz
jeśli
masz
mię
za
podbój
zbyt
łatwy
,
To
zmarszczę
czoło
i
przewrotną
będę
,
I
na
miłosne
twoje
oświadczenia
Powiem
:
nie
,
w
innym
razie
za
nic
w
świecie
.
Za
czuła
może
jestem
,
o
!
Monteki
,
Stąd
możesz
sądzić
me
obejście
płochym
;
Ufaj
mi
jednak
,
będę
ja
wierniejsza
Od
tych
,
co
bieglej
umieją
się
drożyć
.
Byłabym
ja
się
była
,
prawdę
mówiąc
,
Także
drożyła
,
gdybyś
był
tajnego
Głosu
miłości
mojej
nie
podchwycił
.
Nie
wiń
mię
przeto
ani
też
przypisuj
Płochości
tego
wylania
mych
uczuć
,
Które
zdradziła
noc
ciemna
.
ROMEO
O
!
Julio
,
Przysięgam
na
ten
księżyc
,
co
wspaniale
Powleka
srebrem
tamtych
drz

.
MARTA
Jeżeli
on
na
mnie
co
powiedział
,
dam
ja
mu
,
chociażby
był
zuchwalszy
,
niż
jest
,
i
miał
ze
sobą
dwudziestu
sobie
podobnych
drabów
;
a
jeżeli
mi
ujdzie
,
to
znajdę
takich
,
co
to
potrafią
.
A
hultaj
!
czy
to
ja
jestem
jego
kochanką
,
jego
poniewieradłem
!
(
do
Piotra
)
I
ty
tu
stałeś
także
i
mogłeś
ścierpieć
,
żeby
mnie
lada
gbur
używał
wedle
upodobania
za
przedmiot
swych
bezwstydnych
żartów
?
PIOTR
Nie
widziałem
jeszcze
,
żeby
kto
używał
jejmości
wedle
upodobania
;
gdybym
był
to
widział
,
byłbym
był
pewnie
zaraz
giwer
wydobył
,
ręczę
za
to
.
Umiem
się
najeżyć
tak
dobrze
jak
kto
inny
,
kiedy
mam
sposobność
po
temu
i
prawo
za
sobą
.
MARTA
Dlaboga
!
tak
jestem
rozdrażniona
,
że
się
wszystko
we
mnie
trzęsie
.
A
hultaj
!
Otóż
,
proszę
pana
,
tak
jak
powiedziałam
,
młoda
moja
pani
kazała
mi
się
wywiedzieć
o
panu
;
co
mi
kazała
powiedzieć
,
to
sobie
zachowuję
;
ale
przede
wszystkim
oświadczam
panu
,
że
jeżelibyś
ją
osadził
na
koszu
,
jak
to
mówią
,
bo
panienka
,
o
której
mówię
,
je

.
Wchodzą
obywatele
it
d
.
/
PIERWSZY
OBYWATEL
Gdzie
on
?
Gdzie
uszedł
zabójca
Merkucja
?
Zabójca
Tybalt
w
którą
uszedł
stronę
?
BENWOLIO
Tybalt
tu
leży
.
PIERWSZY
OBYWATEL
Za
mną
,
mości
panie
;
W
imieniu
księcia
każęć
być
posłusznym
.
/
Wchodzą
Książę
z
orszakiem
,
Monteki
i
Kapulet
z
małżonkami
swymi
i
inne
osoby
.
/
KSIĄŻĘ
Gdzie
są
nikczemni
sprawcy
tej
rozterki
?
BENWOLIO
Dostojny
książę
,
ja
mogę
objaśnić
Cały
bieg
tego
nieszczęsnego
starcia
.
Oto
tu
leży
,
przez
Romea
zgładzon
,
Zabójca
twego
krewnego
,
Merkucja
.
PANI
KAPULET
Tybalt
!
Mój
krewny
!
Syn
mojego
brata
!
Boże
!
Tak
marnie
zgładzony
ze
świata
!
O
mości
książę
,
błagam
twej
opieki
,
Niech
za
krew
naszą
odda
krew
Monteki
.
KSIĄŻĘ
Benwolio
,
powiedz
,
kto
ten
spór
zapalił
?
BENWOLIO
Tybalt
,
którego
Romeo
powalił
.
Romeo
darmo
przekładał
,
jak
próżną
Była
ta
kłótnia
,
przypominał
zakaz
Waszej
książęcej
mości
,
ale
wszystkie
Te
przedstawienia
,
uczynione
grzecznie
,
Spokojnym
głosem
,
nawet
w
korny
sposób
,
Nie
mogły
wpł

tobą
byłby
zbyt
bolesny
.
Żegnam
cię
,
ojcze
.
/
Wychodzą
.
/
SCENA
CZWARTA
/
Pokój
w
domu
Kapuletów
.
Wchodzą
Kapulet
,
Pani
Kapulet
i
Parys
.
/
KAPULET
Tak
smutny
dotknął
nas
,
panie
,
wypadek
.
Żeśmy
nie
mieli
czasu
mówić
z
Julią
.
Krewny
nasz
,
Tybalt
,
był
jej
nader
drogim
,
Nam
także
,
ale
rodzim
się
,
by
umrzeć
.
Dziś
ona
już
nie
zejdzie
,
bo
już
późno
.
Gdyby
nie
twoje
,
hrabio
,
odwiedziny
,
Ja
sam
bym
w
łóżku
był
już
od
godziny
.
PARYS
Pora
żałoby
nie
sprzyja
zalotom
;
Dobranoc
,
pani
,
poleć
mnie
swej
córce
.
PANI
KAPULET
Najchętniej
,
zaraz
jutro
ją
wybadam
;
Na
dziś
zamknęła
się
,
by
żal
swój
spłakać
.
KAPULET
Hrabio
,
za
miłość
naszego
dziecięcia
Mogę
ci
ręczyć
;
mniemam
,
że
się
skłoni
Do
mych
przełożeń
,
co
więcej
,
nie
wątpię
.
Pójdź
do
niej
,
żono
,
nim
się
spać
położysz
;
Oznajm
jej
cnego
Parysa
zamiary
I
powiedz
–
że
jej
,
uważasz
,
iż
w
środę
…
Zaczekaj
,
cóż
to
dzisiaj
?
PARYS
Poniedziałek
.
KAPULET
A
!
poniedziałek
!
Za
wcześnie
we
środę
;
Odłóżmy
to
na
czwartek


,
Czyli
też
mam
przyjść
wieczór
po
nieszporach
?
OJCIEC
LAURENTY
Nie
brak
mi
teraz
czasu
,
smętne
dziecię
.
Racz
,
panie
hrabio
,
zostawić
nas
samych
.
PARYS
Niech
mię
Bóg
broni
świętym
obowiązkom
Stać
na
przeszkodzie
!
Julio
,
w
czwartek
z
rana
Przyjdę
cię
zbudzić
.
Bądź
zdrowa
tymczasem
I
przyjm
pobożne
to
pocałowanie
.
/
wychodzi
/
JULIA
O
!
zamknij
,
ojcze
,
drzwi
;
a
jak
je
zamkniesz
,
Przyjdź
płakać
ze
mną
.
Nie
ma
już
nadziei
!
Nie
ma
ratunku
!
Nie
ma
ocalenia
!
OJCIEC
LAURENTY
Ach
,
Julio
!
Znam
twą
boleść
;
mnie
samego
Nabawia
ona
prawie
odurzenia
,
Słyszałem
,
i
nic
tego
nie
odwlecze
,
Że
w
przyszły
czwartek
wziąć
masz
ślub
z
tym
hrabią
JULIA
Nie
mów
mi
,
ojcze
,
że
o
tym
słyszałeś
;
Chyba
,
że
powiesz
,
jak
tego
uniknąć
,
Jeżeli
w
swojej
mądrości
nie
znajdziesz
Żadnego
na
to
środka
,
to
przynajmniej
Postanowienie
moje
nazwij
mądrym
,
A
w
tym
sztylecie
zaraz
znajdę
środek
.
Bóg
złączył
moje
i
Romea
ręce
,
Ty
nasze
dłonie
;
i
nim
ta
dłoń
,
świętą
Pieczęcią
twoją
z
Romeem
spojo

Wchodzi
Baltazar
.
/
Wieści
z
Werony
!
-
Cóż
tam
,
Baltazarze
?
Czy
mi
przynosisz
list
od
Laurentego
?
Co
robi
Julia
?
Czy
zdrów
jest
mój
ojciec
?
Jak
się
ma
Julia
?
Po
raz
drugi
pytam
.
Bo
nie
ma
złego
,
jeśli
jej
jest
dobrze
.
BALTAZAR
Wszystko
więc
dobrze
,
bo
jej
już
źle
nie
jest
;
Ciało
jej
leży
w
lochach
Kapuletów
,
A
duch
jej
gości
między
aniołami
.
Widziałem
,
jak
ją
złożono
do
sklepień
,
I
wziąłem
pocztę
,
aby
o
tym
panu
Donieść
czym
prędzej
.
Przebacz
pan
,
że
taką
Złą
wieść
przynoszę
;
wszakże
uwiadomić
Pana
o
wszystkim
byłem
w
obowiązku
.
ROMEO
Maż
to
być
prawdą
?
Drwię
sobie
z
was
,
gwiazdy
!
Wszak
wiesz
,
gdzie
mieszkam
?
Przynieś
mi
papieru
I
atramentu
,
idź
potem
na
pocztę
Zamienić
konie
.
Wyjeżdżam
tej
nocy
.
BALTAZAR
Błagam
cię
,
panie
,
zachowaj
cierpliwość
;
Wyglądasz
blado
,
ponuro
i
wzrok
twój
Coś
niedobrego
zapowiada
.
ROMEO
Cicho
.
Mylisz
się
;
zostaw
mię
,
zrób
,
com
rozkazał
.
Czy
nie
masz
listu
od
księdza
?
BALTAZAR
Nie
,
panie
.
ROMEO
Mniejsza
więc
o
to
.
Id

napotkacie
.
/
Wychodzi
kilku
ludzi
z
warty
.
/
Smutny
widoku
!
tu
hrabia
zabity
,
Tu
Julia
we
krwi
pływa
,
jeszcze
ciepła
,
Tylko
co
zmarła
;
ona
,
co
przed
dwoma
Dniami
w
tym
grobie
była
pochowana
.
Idźcie
powiedzieć
o
tym
księciu
;
śpieszcie
,
Wy
do
Montekich
,
wy
do
Kapuletów
,
A
wy
odbądźcie
przegląd
w
innej
stronie
.
/
Wychodzi
kilku
innych
wartowników
.
/
Widzimy
miejsce
,
gdzie
zaszła
ta
zgroza
,
Lecz
w
jaki
sposób
ona
miała
miejsce
,
Tego
nie
możem
pojąć
bez
objaśnień
.
/
Wchodzi
kilku
innych
wartowników
z
Baltazarem
.
/
PIERWSZY
WARTOWNIK
Oto
Romea
sługa
,
znaleźliśmy
Go
na
cmentarzu
.
DOWÓDCA
Niech
będzie
pod
strażą
,
Dopóki
książę
nie
nadejdzie
.
/
Wchodzi
kilku
innych
wartowników
,
prowadząc
Ojca
Laurentego
.
/
DRUGI
WARTOWNIK
Oto
mnich
jakiś
drżący
i
płaczący
;
Odebraliśmy
mu
ten
drąg
i
rydel
,
Kiedy
się
bokiem
cmentarza
wykradał
.
DOWÓDCA
To
jakiś
ptaszek
;
trzymajcie
go
także
.
/
Wchodzi
Książę
ze
swym
orszakiem
.
/
Co
za
nieszczęście
o
tak
rannej
porze
Sen
nasz
przerwa

# 6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [79]:
import random
import time

def removeTokens(tokens,percentage):
    random.seed(time.time())
    to_delete = [random.choice(tokens) for i in range(int(len(tokens)*percentage/100))]
    print("Usunięte tokeny:")
    print(str(to_delete) + "\n\n")
    return [token for token in tokens if token not in to_delete]

In [80]:
cut_tokenized_data_1 = removeTokens(tokenized_data,3)
print("Procent usuniętych tokenów: " + str((len(tokenized_data)-len(cut_tokenized_data_1))/(len(tokenized_data))*100) + "%")

Usunięte tokeny:
[że, To, zawczasu, cię, ,, ,, tu, /, ., Kształtna, Tak, ,, OJCIEC, Nim, zaradzić, ściśle, Ha, ,, :, ,, ;, kto, kornej, nianiunieczku, w, staną, mi, ,, paź, ., leżąc, przynajmniej, że, tu, moje, odejść, kładzie, Powinna, !, i, kropel, !, —, Zbierz, ,, ten, poplecznikami, ,, gdybym, mój, swego, żebraczej, w, Wciąż, jakby, z, czy, tak, on, Grzegorz, znał, mojej, jej, Wchodzi, boże, Miłość, !, sędziwy, jaki, ,, ., jak, Jak, więcej, krwią, stokroć, /, ma, /, Wolne, lekką, pojąć, do, ta, ,, Wchodzą, upatrzysz, przebraniu, BENWOLIO, Przeciął, Hrabio, panie, ., O, w, sama, BENWOLIO, !, w, Kłamiesz, znajduje, ona, Lontem, tu, i, Bardzo, Tak, KAPULET, ,, LAURENTY, gałązce, Czy, jak, ,, ., naszego, PANI, JULIA, To, ,, Ha, ,, je, uroku, w, zdrów, truciznę, Wesołe, Julia, /, pan, Tak, stokroć, losem, mówiąc, szukasz, samo, się, żywotem, :, ci, inaczej, nieczysta, Puść, o, mordu, obiad, Wchodzą, !, nocą, chmurami, nie, się, monetą, Tylko, w, jakże, nocy, JULIA, ?, praw, zapewnień, ż

In [81]:
cut_tokenized_data_2 = removeTokens(tokenized_data,3)
print("Procent usuniętych tokenów: " + str((len(tokenized_data)-len(cut_tokenized_data_1))/(len(tokenized_data))*100) + "%")

Usunięte tokeny:
[Szpetny, nie, Romeo, ,, powie, Jednego, go, mego, pani, ., żebyś, mej, ;, ,, miał, !, ., właściwej, ?, głowę, piękny, tego, I, się, Jedną, zwierzęta, jedynie, ,, ,, pod, *, ty, się, :, wszakże, smok, rzępołów, ją, /, nie, !, pacierz, grobowca, ,, wyzwaniem, ., Przeciął, czas, !, Mogę, wnet, sosem, mieć, ,, walcząc, !, Sługi, ;, zmorą, ty, łez, Ja, na, krok, Spłyńcie, gorzką, sobie, spać, Tym, w, muszę, mieć, ., to, już, świadkiem, było, ROMEO, „, Nad, mój, Będę, to, to, ., nasze, droga, A, ., Ach, SCENA, sobie, Niewielkie, pokrytej, jej, czułą, miłość, nieszczęsnej, zgody, jedno, BRAT, ,, sobie, spotkaniu, nie, Boże, ,, ,, wybrnę, Wołają, szalony, !, BALTAZAR, i, Wychodzą, To, –, PARYS, mur, Oto, dla, wasze, z, śmierć, ., przestać, Syn, godzinie, Tybalt, złączmy, ,, wyzwanie, jak, Przyjdzie, Jeśli, także, noszą, majątków, do, cię, jak, zawiódł, te, żalowi, zabił, !, ludzie, ;, jeszcze, Nim, razem, ., ,, to, JULIA, W, jak, żebyś, Gdyby, Romeo, /, odda, trzewików, ,, .,

Upewnijmy się, że stworzone tokeny to nie te same zbiory (gdyby random.seed zawiódł)

In [82]:
print(cut_tokenized_data_1 == cut_tokenized_data_2)

False


Jest więc OK :)

# 7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.

In [83]:
tokens_lcs = lcs(cut_tokenized_data_1,cut_tokenized_data_2)

In [87]:
print("Długość najdłuższego podciągu wspólnych tokenów jest równa " + str(tokens_lcs))
print("Jakim procentem wszystkich tokenów jest LCS? " + str(tokens_lcs/(len(tokenized_data))*100) + "%")

Długość najdłuższego podciągu wspólnych tokenów jest równa 26274
Jakim procentem wszystkich tokenów jest LCS? 94.19229941922994%


# Wniosek

Wynik nie powinien dziwić. Ponieważ z obydwu tekstów usunęliśmy około 3% tokenów i możemy z dużym prawdopodobieństwem przyjąć, że są one w większości od siebie różne, to niemal wszystkie tokeny tworzą najdłuższy wspólny podciąg - w końcu wszystkie tokeny na początku były takie same. Jedynie około 2*3% = 6% tokenów nie powinno tworzyć LCS (te które usunęlismy). Zatem wynik okołu 94% jest jak najbardziej poprawny oraz łatwy do przewidzenia.

# 8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.

Na początek przygotuję sobie funkcję, która otwiera plik i zwraca go jako tablice stringów, na której łatwiej będzie operować

In [95]:
# funkcja otwierająca plik tekstowy oraz zwracająca go jako tablica stringów
def read_from_file(file1):
    data = []
    with open(file1,mode='r', encoding="utf-8") as f:
        for line in f:
            data.append(line.strip())
    f.close()
    return data

Teraz wprowadzę algorytm z zadania 4 zmodyfikowany na potrzeby naszego zadania

In [138]:
# metoda do znajdowania zbędnych elementów w LCS
def lcs_diff(f1,f2):
    s = read_from_file(f1)
    t = read_from_file(f2)
    
    a = len(s)
    b = len(t)
    
    lcs_table = [[0] * (b+1) for i in range(a+1)]
    
    for i in range(a+1):
        for j in range(b+1):
            if i == 0 or j == 0:
                lcs_table[i][j] = 0
            elif s[i-1] == t[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i-1][j],lcs_table[i][j-1])
    
    
    sequence = []
    m,n = len(s),len(t)

    while m != 0 or n != 0:
        if s[m-1] == t[n-1]:
            sequence.append(s[m-1])
            m,n = m-1,n-1
        elif lcs_table[m-1][n] > lcs_table[m][n-1]:
            m -= 1
        else:
            n -= 1
    
    sequence.reverse()
    
    i,j = 0,0
    
    print("Znaczenie symboli (numerowanie linii od 1):")
    print("< i | linia i pliku " + str(f1) + " nie nalezy do LCS")
    print("> i | linia i pliku " + str(f2) + " nie nalezy do LCS")
    print("LCS to " + str(sequence) + "\n\n")
    
    for line in sequence:
        while (i < len(s) and s[i] != line) or (j < len(t) and t[j] != line):
                        
            if i < len(s) and s[i] != line:
                print("< " + str(i+1))
                i += 1
            if j < len(t) and t[j] != line:
                print("> " + str(j+1))
                j += 1
            
        i,j = i+1,j+1
    
    while i < len(s):
        print("< " + str(i+1))
        i += 1
    
    while j < len(t):
        print("> " + str(j+1))
        j += 1

### Przykład działania na plikach tekst1.txt oraz tekst2.txt załączonych w zadaniu

In [143]:
print(read_from_file("tekst1.txt"))
print(str(read_from_file("tekst2.txt")) + "\n\n")
lcs_diff("tekst1.txt","tekst2.txt")

['ala', 'ma', 'kota', 'a', 'kot', 'ma', 'ale']
['ala', 'idzie', 'do', 'psa', 'a', 'pies', 'ma', 'kota', 'szaroburego', 'ale', 'fajnie', 'jest']


Znaczenie symboli (numerowanie linii od 1):
< i | linia i pliku tekst1.txt nie nalezy do LCS
> i | linia i pliku tekst2.txt nie nalezy do LCS
LCS to ['ala', 'a', 'ma', 'ale']


< 2
> 2
< 3
> 3
> 4
< 5
> 6
> 8
> 9
> 11
> 12


# 9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

Na początku przygotuję dwa pliki tokens1.txt oraz tokens2.txt, które w każdej kolejnej linii będą zawierały kolejne tokeny. Zadbam również o to, żeby tokeny faktycznie były w kolejnych liniach tj. dodam znaki przejścia do nowej linii.

In [145]:
with open("tokens1.txt",mode='w', encoding="utf-8") as f:
    for token in cut_tokenized_data_1:
        f.write(str(token)+"\n")
f.close()

In [146]:
with open("tokens2.txt",mode='w', encoding="utf-8") as f:
    for token in cut_tokenized_data_2:
        f.write(str(token)+"\n")
f.close()

Teraz zostało mi jedynie wywołać algorytm stworzony w zadaniu 8 dla tych plików:

In [147]:
lcs_diff("tokens1.txt","tokens2.txt")

Znaczenie symboli (numerowanie linii od 1):
< i | linia i pliku tokens1.txt nie nalezy do LCS
> i | linia i pliku tokens2.txt nie nalezy do LCS
LCS to ['William', 'Shakespeare', 'Romeo', 'i', 'tłum', '.', 'Józef', 'Paszkowski', 'ISBN', '978', '-', '83', '-', '-', '2903', '-', '9', 'OSOBY', ':', '*', 'ESKALUS', '—', 'książę', 'panujący', 'w', 'Weronie', 'PARYS', '—', 'młody', 'Weroneńczyk', 'szlachetnego', 'rodu', ',', '*', 'MONTEKI', ',', 'KAPULET', 'naczelnicy', 'dwóch', 'domów', 'nieprzyjaznych', 'sobie', '*', 'STARZEC', '—', 'stryjeczny', 'brat', 'Kapuleta', '*', 'ROMEO', '—', 'syn', 'Montekiego', '*', 'MERKUCJO', '—', 'krewny', 'księcia', '*', 'BENWOLIO', '—', 'synowiec', 'Montekiego', 'TYBALT', '—', 'krewny', 'Pani', 'Kapulet', '*', 'LAURENTY', '—', 'ojciec', 'franciszkanin', '*', 'JAN', '—', 'brat', 'z', 'tegoż', 'zgromadzenia', '*', 'BALTAZAR', '—', 'służący', 'Romea', '*', 'SAMSON', 'GRZEGORZ', '—', 'słudzy', 'Kapuleta', '*', 'ABRAHAM', '—', 'służący', 'Montekiego', '*', 'APTEK

> 23625
< 23632
< 23642
< 23652
> 23667
< 23697
> 23696
> 23698
> 23700
< 23730
> 23731
> 23733
< 23757
> 23780
> 23781
< 23789
> 23818
< 23834
> 23849
> 23876
< 23888
> 23918
> 23923
< 23924
< 23929
> 23942
< 23982
> 23985
> 23990
< 24011
> 24035
< 24040
> 24048
< 24051
> 24052
> 24078
> 24086
< 24103
< 24124
> 24154
< 24216
> 24280
< 24288
> 24296
< 24327
< 24348
> 24356
< 24367
< 24402
< 24454
> 24461
> 24464
< 24492
< 24511
> 24524
< 24532
> 24551
< 24554
> 24567
< 24592
< 24593
< 24596
> 24605
> 24626
> 24650
< 24656
> 24660
> 24701
> 24737
> 24777
< 24794
< 24810
> 24824
< 24837
< 24843
> 24843
> 24856
< 24857
< 24871
< 24882
< 24899
> 24959
< 24972
< 24981
< 24993
> 24994
< 25008
> 25014
< 25024
> 25025
< 25037
> 25033
< 25046
< 25096
> 25091
< 25130
> 25157
> 25182
> 25192
< 25259
> 25311
> 25328
< 25344
> 25372
> 25385
> 25390
> 25409
< 25421
< 25422
< 25430
> 25430
> 25448
< 25468
> 25477
< 25492
< 25501
< 25529
< 25552
< 25587
< 25626
< 25636
< 25640
> 25633
> 25681
> 25682


Można łatwo sprawdzić, że początkowe linie wypisane przez algorytm faktycznie się zgadzają (nie należą do LCS). Porównując kilka pierwszych wyników oraz wyniki porównań z zadaniu 8 można stwierdzić, że algorytm jest poprawny i działa tak jak należy

# Podsumowanie

Przekrój zadań dotyczących odległości edycyjnej został przez Pana Doktora wybrany w interesujący oraz mocno dydaktyczny sposób.
Wykonane zadanie pozwoliło się dobrze zapoznać z pojęciem odległości edycyjnej, gdzie tego można użyć oraz jak i z czym to się je. Ćwiczenie pozwoliło również dowiedzieć się kilku ciekawych rzeczy dotyczących odległości edycyjnej oraz silnego narzędzia jakim jest spaCy. Wykonane laboratorium było przyjemne, ciekawe oraz satysfakcjonujące. Dziękuje!